# Sparse Portfolio Stat Arb with Ornstein Uhlenbeck Spread Calculation

### Equity Stat Arb Strategy

At a super high level, this strat involves finding a bunch of stocks from the same sector, calculating weights to create a mean reverting portfolio, then trading the spread of that portfolio.

Steps:

1. **Asset Universe Selection**  
   Select a basket of N correlated equities (e.g., 50 stocks from the same sector in the S&P 500). This maximizes the chance of finding stationary relationships in the price data.

2. **Data Collection**  
   Collect intraday historical OHLCV data (e.g., 1h bars) for the selected tickers. Use log prices as the input for portfolio construction and spread modeling.

3. **Create Sparse Mean Reverting Portfolio**  
   Solve a convex optimization problem that finds a sparse weight vector `w` that minimizes the OU negative log-likelihood of the spread `X_t = w^T * log(P_t)`, subject to an L1 penalty on `w`.  
   - **Input**: Log price matrix of shape `(T, N)`  
   - **Output**: Sparse portfolio weights `w` of length `N`

4. **Fit an OU Model to the Spread**  
   Model the spread `X_t` using a discrete-time Ornstein-Uhlenbeck process:  
   `X_{t+1} = mu + (1 - theta) * (X_t - mu) + epsilon_t`, where `epsilon_t ~ N(0, sigma^2)`  
   Estimate OU parameters `(mu, theta, sigma)` using maximum likelihood estimation.  
   - **Input**: Spread time series `X_t`  
   - **Output**: OU parameters `mu`, `theta`, and `sigma`

5. **Per Tick: Compute Spread, Compute Z-score, Generate Signals**  
   For each new price observation:  
   - Recompute `X_t = w^T * log(P_t)`  
   - Compute z-score from OU model:  
     `Z_t = (X_t - mu) / (sigma / sqrt(2 * theta))`  
   - Generate trading signals:  
     - **Long** if `Z_t < -entry_threshold`  
     - **Short** if `Z_t > entry_threshold`  
     - **Exit** if `|Z_t| < exit_threshold`  
   - **Output**: Signal object with position direction and notional weights
